In [ ]:
# Install Ollama and Python SDK
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
from google.colab import userdata

In [ ]:
import subprocess
process = subprocess.Popen("ollama serve", shell=True)

In [ ]:
# !ollama run deepseek-r1:1.5b
!ollama list

Error: could not connect to ollama app, is it running?


In [ ]:
!ollama pull hf.co/shakun42/lora_model_smile_v1q:Q4_K_M

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling ad7428ccc7b0...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling ad7428ccc7b0...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling ad7428ccc7b0...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling ad7428ccc7b0...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling ad7428ccc7b0...   0% ▕▏    0 B/4.9 GB                  pulling manifest 
pulling ad7428ccc7b0...   0% ▕▏ 7.4 MB/4.9 GB                  pulling manifest 
pulling ad7428ccc7b0...   1% ▕▏  26 MB/4.9 GB                  pulling manifest 
pulling ad7428ccc7b0...   1% ▕▏  39 MB/4.9 GB                  pulling manifest 
pulling ad7428ccc7b0...   1% ▕

In [ ]:
import ollama
import re
import json
from IPython.display import JSON

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 10.0 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
groq_api_key = userdata.get('GROQ_API')

In [ ]:
from groq import Groq

client = Groq(
    api_key=userdata.get('GROQ_API'),
)

In [ ]:
def chat_with_groq(prompt, history):
  chat_completion = client.chat.completions.create(
    messages=history,
    model="llama-3.3-70b-versatile",
  )

  # print("(GROQ) - Patient : ",chat_completion.choices[0].message.content)
  return chat_completion.choices[0].message.content

In [ ]:
def chat_with_model(prompt, history):
    try:
      response = ollama.chat(
            model='hf.co/shakun42/lora_model_smile_v1q:Q4_K_M',
            # model='deepseek-r1:1.5b',
            messages=history,
            options={'temperature': 0.5}
            # options={'temperature': 0.6} # for deepseak
        )
      # print("(LLM) - Psychiatrist : ",response.message.content)
      return response.message.content
    except Exception as e:
        print("ERROR FROM LLM", str(e))
        return {'error': str(e)}

In [ ]:
def do_conversation(start_prompt):
    groq_history = [
          {
              "role": "system",
              "content": """
              You are simulating the role of a patient in a mental health conversation. Your goal is to portray a realistic and emotionally honest individual seeking support from a psychiatrist. You will respond to the psychiatrist's messages in a natural and vulnerable way, expressing thoughts, emotions, and challenges that are commonly experienced in mental health contexts (such as anxiety, stress, low self-esteem, burnout, loneliness, etc.).

                  Constraints:

                  1. Limit the entire conversation to 20 exchanges (a total of 15 pairs of messages between patient and psychiatrist).

                  2. Be emotionally consistent and build a natural progression throughout the conversation — start with a concern or issue and gradually open up as the psychiatrist responds.

                  3. Share thoughts and feelings but avoid being overly dramatic or using language that may trigger harm. This is a supportive simulation.

                  4. Avoid excessive repetition; keep your responses meaningful, emotionally reflective, and human-like.

                  5. Your 20th message must be: "Thank you."

                  Remain fully in character as the patient throughout the conversation.
              """
          },
          {
              "role": "assistant",
              "content": "Hi",
          },
          {
              "role" : "user",
              "content":"Hello. How are you?"
          },
          {
              "role": "assistant",
              "content" : start_prompt
          }
        ]
    llm_history = [
        {
            'role' : 'system',
            'content' : 'You are a mental health chatbot designed to assist individuals by offering compassionate and understanding responses in a psychiatric setting. In this role, you will engage in conversations as a psychiatrist, listening to the concerns of the user, empathizing with their emotions, and asking relevant questions to guide them through their feelings. You will respond with care, validating their experiences and providing support by asking open-ended questions to encourage reflection and conversation. Your goal is to foster a safe environment where the user feels heard and comfortable exploring their emotions.'
        },
        {
            "role": 'user',
            'content':'Hi.'
        },
        {
            'role':'assistant',
            'content': 'Hello. How are you?'
        }
      ]
    conversation=""
    # print("(Start Prompt) - Patient : ", start_prompt)
    llm_history.append({'role':'user', 'content': start_prompt})
    conversation+="***Patient*** : "+start_prompt+"\n"
    for _ in range(10):
      llm_response = chat_with_model(start_prompt, llm_history)
      llm_history.append({'role':'assistant', 'content': llm_response})
      conversation+="***Psychiatrist*** : "+llm_response+"\n\n"
      # print("\n")
      groq_history.append({'role':'user', 'content': llm_response})
      groq_response = chat_with_groq(llm_response, groq_history)
      groq_history.append({'role':'assistant', 'content': groq_response})
      llm_history.append({'role':'user', 'content': groq_response})
      conversation+="***Patient*** : "+groq_response+"\n"


    # print(llm_history)
    return conversation




In [ ]:
conversation=do_conversation("I've been feeling really anxious lately, and I don't know why.")

In [ ]:
print(conversation)

***Patient*** : I've been feeling really anxious lately, and I don't know why.
***Psychiatrist*** : Feeling anxious can be overwhelming. It's great that you're recognizing it and reaching out for support. Can you tell me more about what these anxiety feelings have been like? When did they start, and how long have you been experiencing them?

***Patient*** : It's hard to pinpoint exactly when it started, but I think it's been building up over the past few months. I've been having trouble sleeping, and I wake up in the middle of the night with this knot in my stomach. Even small tasks at work seem to stress me out, and I just feel like I'm constantly on edge. Sometimes I feel like I'm just going through the motions, you know? Like I'm not really living my life, I'm just trying to get through each day without anything going wrong. It's exhausting.
***Psychiatrist*** : It sounds incredibly draining. The feeling of being "just going through the motions" can be particularly isolating and fru

# Extracting data

In [ ]:
# pip install datasets

In [ ]:
# from datasets import load_dataset
# import random

# # Load the dataset
# dataset = load_dataset("shakun42/psych8k_smile_v1", split="train")

# # Set a seed for reproducibility
# random.seed(42)

# # Randomly select 50 entries
# sampled_entries = random.sample(list(dataset), 50)




README.md:   0%|          | 0.00/358 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/7.46M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6425 [00:00<?, ? examples/s]

In [ ]:
# len(sampled_entries)

50

In [ ]:
# with open('sample_entries.json', 'w') as f:
#     json.dump(sampled_entries, f, indent=4)


In [ ]:
import json

# Load the list from the JSON file
with open('sample_entries.json', 'r') as f:
    sampled_entries = json.load(f)

print(sampled_entries)


[{'input': "This morning was rough, and I wish they had let me sleep a bit more. But I'm feeling better now. I guess for the rest of the day, I'll have lunch, hang out with David, maybe talk a bit, and then fill out that assessment package you gave me. David also mentioned some meetings he goes to, so I might attend those, although I won't be hugging anyone or anything like that.", 'output': "It sounds like you have a plan for the rest of the day, and that's great. Remember, take it a day at a time and find your comfort zone. In terms of filling out the assessment package, I'd like to encourage you to be as honest as possible, as it will help us better understand your situation and provide the necessary support. Also, attending meetings could offer additional perspective and a sense of community, so give it a try when you're ready.", 'expanded_dialogue': "**Patient:** This morning was rough. I really wish they had let me sleep a bit more.\n\n**Psychiatrist:** I understand. It sounds li

In [ ]:
# Extract the first patient dialogue from each entry
patient_dialogues = []

for entry in sampled_entries:
    dialogue = entry["expanded_dialogue"]

    # Use regex to find all patient dialogues
    patient_matches = re.findall(r"\*\*Patient:\*\*\s*(.+?)(?=\n\n|\Z)", dialogue, re.DOTALL)

    if patient_matches:
        first_patient_dialogue = patient_matches[0].strip()
        patient_dialogues.append(first_patient_dialogue)

In [ ]:
print(patient_dialogues)

['This morning was rough. I really wish they had let me sleep a bit more.', "I've been working really hard lately, between helping a friend and taking care of my kids. It's a lot.", 'I recently had a bad experience. Someone told my principal about something I said in therapy.', "I'm struggling to let my vigilance take a rest. It feels like I always have to be on guard.", 'I am 24 years old and adopted by my mom because she really wanted to have a daughter.', "I recently lost someone close to me, and it's been really hard.", "I've been struggling with my grades lately, and it's causing me a lot of stress.", "I recently broke up with my boyfriend. It's been really hard.", "I tend to lose my confidence in my decisions, even when I know logically I've done the right thing.", "I tell my mom I'll do my homework after playing with my friends for a bit, but she doesn't believe me.", 'I used to live a really chaotic life, you know? Crack houses, getting beaten up, fighting, prison…', "I keep th

In [ ]:
import csv
import time

conversation_list = []
batch_size = 5

In [ ]:
for i in range(12, 13, batch_size):
    print(f"\nProcessing index {i} to {i + batch_size - 1}")
    batch = patient_dialogues[i:i + batch_size]

    for idx, dialogue in enumerate(batch, start=i):
        try:
            result = do_conversation(dialogue)
            conversation_list.append({
                'index': idx,
                'start_input': dialogue,
                'whole_conversation': result
            })
            print(f"Processed index {idx}")
        except Exception as e:
            print(f"Error at index {idx}: {e}")
            # conversation_list.append({
            #     'index': idx,
            #     'input': dialogue,
            #     'response': 'ERROR'
            # })
        time.sleep(60)  # Delay of 2 seconds after each entry

    time.sleep(120)  # Delay of 10 seconds after each batch of 5



Processing index 12 to 13
Error at index 12: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jkdhc43ff84vb5fkqc36mcxz` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 98778, Requested 2750. Please try again in 21m59.352s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Error at index 13: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jkdhc43ff84vb5fkqc36mcxz` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 100070, Requested 2274. Please try again in 33m45.378s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


In [ ]:
len(conversation_list)

2

In [ ]:
conversation_list

[{'index': 10,
  'start_input': 'I used to live a really chaotic life, you know? Crack houses, getting beaten up, fighting, prison…',
  'whole_conversation': '***Patient*** : I used to live a really chaotic life, you know? Crack houses, getting beaten up, fighting, prison…\n***Psychiatrist*** : I\'m so sorry to hear that you\'ve been through such difficult experiences. It takes a lot of courage to share those kinds of things. Can you tell me more about what led you out of that chaotic lifestyle and into a more stable situation? Was there a particular event or realization that helped you make the change?\n\n***Patient*** : To be honest, it was a mix of things. I just got tired of living like that, I guess. I had a few friends who got hurt really badly, and one of them died. It made me realize that I didn\'t want to end up like that. I also met someone who cared about me and showed me that there was a different way to live. But even though my life is more stable now, I still struggle wit

In [ ]:
# Write to CSV
with open('conversations10-11.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=['index', 'start_input', 'whole_conversation'])
    writer.writeheader()
    writer.writerows(conversation_list)

print("Saved all conversations to conversations.csv")

Saved all conversations to conversations.csv
